In [1]:
import psycopg2
import geopandas as gpd
from collections import namedtuple
from psycopg2 import sql

In [2]:
pwfile = open("./postgis/pwfile")
pw = pwfile.readline().strip()
pwfile.close()

# had to set db auth to scram because md5 bad
connection = psycopg2.connect(host='192.168.1.111', port=5432, dbname='d4d', user='d4d', password='%s' % pw)
cur = connection.cursor()

In [3]:
cur.execute("SELECT column_name FROM information_schema.columns WHERE TABLE_NAME='iraq_points'")
cur.fetchall()

[('ogc_fid',),
 ('shape_orientation',),
 ('detect_type',),
 ('iids',),
 ('change_area',),
 ('weighted_centroid_lat',),
 ('target_minimum_return',),
 ('background_minimum_return',),
 ('ref_graze_angle',),
 ('processing_type',),
 ('target_maximum_return',),
 ('sec_azimuth_angle',),
 ('ref_azimuth_angle',),
 ('shape_extent_minor',),
 ('weighted_centroid_lon',),
 ('background_maximum_return',),
 ('shape_extent_major',),
 ('perimeter',),
 ('pixel_count',),
 ('timestamp',),
 ('centroid_lat',),
 ('target_total_return',),
 ('detect_strength',),
 ('eccentricity',),
 ('centroid_lon',),
 ('return_units',),
 ('name',),
 ('algorithm',),
 ('target_mean_return',),
 ('sec_graze_angle',),
 ('background_mean_return',),
 ('compactness',),
 ('wkb_geometry',)]

In [4]:
cur.execute("SELECT MAX(ST_X(wkb_geometry)) AS max_long, MAX(ST_Y(wkb_geometry)) AS max_lat FROM iraq_points")
max_long, max_lat = cur.fetchone()
cur.execute("SELECT MIN(ST_X(wkb_geometry)) AS min_long, MIN(ST_Y(wkb_geometry)) AS min_lat FROM iraq_points")
min_long, min_lat = cur.fetchone()

In [5]:
Point = namedtuple('Point', ['lat', 'long'])
Rectangle = namedtuple('Rectangle', ['top_left', 'bottom_left', 'bottom_right', 'top_right'])

def make_subrect(top_left, lat_step, long_step):
    top_right = Point(lat=top_left.lat + lat_step, long=top_left.long)
    bottom_left = Point(lat=top_left.lat, long=top_left.long + long_step)
    bottom_right = Point(lat=top_left.lat + lat_step, long=top_left.long + long_step)
    
    return Rectangle(top_left=top_left, bottom_left=bottom_left, bottom_right=bottom_right, top_right=top_right)

def make_subrects(min_lat, min_long, max_lat, max_long, lat_count=5, long_count=5):
    lat_step = (max_lat - min_lat) / lat_count
    long_step = (max_long - min_long) / long_count
    
    subrects = []
    
    for lat_row in range(lat_count):
        for long_col in range(long_count):
            top_left = Point(lat=min_lat + (lat_row*lat_step), long=min_long + (long_col * long_step))
            subrects.append(make_subrect(top_left, lat_step, long_step))
    
    return subrects

def postgis_polygon_from_rect(rect):
    return """
    ST_MakePolygon( ST_GeomFromText('LINESTRING({tl_lat} {tl_long}, {bl_lat} {bl_long}, {br_lat} {br_long}, {tr_lat} {tr_long}, {tl_lat} {tl_long})', 4326))""".strip(
    ).format(
        tl_lat=rect.top_left.lat,
        tl_long=rect.top_left.long,
        bl_lat=rect.bottom_left.lat,
        bl_long=rect.bottom_left.long,
        br_lat=rect.bottom_right.lat,
        br_long=rect.bottom_right.long,
        tr_lat=rect.top_right.lat,
        tr_long=rect.top_right.long,
    )

In [6]:
rects = make_subrects(min_lat, min_long, max_lat, max_long)
len(rects)

25

In [7]:
choice = rects[12]

In [8]:
query = sql.SQL("""
SELECT AVG(eccentricity) FROM iraq_points WHERE ST_Within(ST_FlipCoordinates(wkb_geometry), {polygon})
""".strip().format(
    polygon= postgis_polygon_from_rect(choice)
))
print(query.as_string(connection))
cur.execute(query)
cur.fetchall()

SELECT AVG(eccentricity) FROM iraq_points WHERE ST_Within(ST_FlipCoordinates(wkb_geometry), ST_MakePolygon( ST_GeomFromText('LINESTRING(32.25302488 42.61423528, 32.25302488 45.59361672, 34.59259722 45.59361672, 34.59259722 42.61423528, 32.25302488 42.61423528)', 4326)))


[(0.825022560969856,)]